In [1]:
# rwr 조화평균 완료

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean

start = time.time()

def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

def calculate_standard_deviation(numbers):
    return statistics.stdev(numbers)

def combine_lists(full_list):
    combined = defaultdict(list)
    
    # 모든 리스트에 대한 처리
    for sublist in full_list:
        for item in sublist:
            entrez_id, rwr_score = item
            combined[entrez_id].append(rwr_score)
    
    # 조화평균 및 결과 리스트 만들기
    result = []
    for entrez_id, scores in combined.items():
        if len(scores) == 1:
            result.append([entrez_id, scores[0]])  # 값이 하나면 그대로 추가
        else:
            # 조화평균 계산
            harmonic_mean_score = harmonic_mean(scores)
            result.append([entrez_id, harmonic_mean_score])
    
    return result

def compound_target(compoundid):

    conn_cur.execute(f"with tmp1 as (select cg.geneid from compound_gene cg where (cg.compid = '{compoundid}' and cg.relationsign in ('activate','inhibit')))" 
                    f"select g.entrezid from gene g,tmp1 where g.geneid=tmp1.geneid;")
    compound_gene1=conn_cur.fetchall()

    compound_gene=set(compound_gene1)
    compound_gene=list(compound_gene)
    
    # Compound-target gene List 생성
    target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            target_gene_list.append(interaction_gene)        
        else:
            continue

    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in target_gene_list:
        node_ = node_index.get(entrez) 
        target_node.append(node_)

    conn_cur.execute(f"with tmp2 as (select cg.geneid from compound_gene cg where cg.compid = '{compoundid}' and relationsign='associate')" 
                    f"select g.entrezid from gene g,tmp2 where g.geneid=tmp2.geneid;")

    compound_gene2=conn_cur.fetchall()

    compound_gene=set(compound_gene2)
    compound_gene=list(compound_gene)

    indirected_target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            indirected_target_gene_list.append(interaction_gene)        
        else:
            continue      

    num1=len(target_gene_list) #target gene list 갯수
    directed_target_gene_list.append(num1)
    num2=len(indirected_target_gene_list) #indirected target gene list 갯수 
    indirected_target_gene_list_count.append(num2)
    
    print("Compound",compoundid,"Target gene 개수",num1)
    print("Compound",compoundid,"indirected_Target gene 개수",num2)
    
    #indirect nodes 정의
    indirect_nodes= []
    for entrez in indirected_target_gene_list:
        node_ = node_index.get(entrez) 
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 

    # RWR 실행
    rwr = RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    
    var_name1 = f'{compoundid}_num1'
    var_name2 = f'{compoundid}_num2'

    # 전역 변수 설정
    globals()[var_name1] = num1
    globals()[var_name2] = num2
    
    return rwr_mapping_entrez

def rwr_compound_target(count,entrez_list):
    
    num1=directed_target_gene_list[count]
    num2=indirected_target_gene_list_count[count]
    
    random_interaction_gene1 = random.sample(entrez_list, num1)
    remaining_entrez_list = [gene for gene in entrez_list if gene not in random_interaction_gene1]
    random_interaction_gene2 = random.sample(remaining_entrez_list, num2)
# restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in random_interaction_gene1:
        node_ = node_index.get(entrez)
        target_node.append(node_)

    #indirect nodes 정의
    indirect_nodes= []
    for entrez in random_interaction_gene2:
        node_ = node_index.get(entrez)
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 
    
# RWR 실행
    rwr=RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    return rwr_mapping_entrez

def create_ppi_network(network_data):
    network_data = network_data.astype({'Entrez Gene Interactor A': str,
                                        'Entrez Gene Interactor B': str})

    # symbol list를 생성하고 unique 값만 가지도록 생성
    symbolA = network_data.loc[:, 'Entrez Gene Interactor A'].to_list()
    symbolB = network_data.loc[:, 'Entrez Gene Interactor B'].to_list()

    symbol_list = symbolA + symbolB 
    symbol_list = set(symbol_list) 
    symbol_list = list(symbol_list)

    # 모든 symbol에 index를 딕셔너리에서 부여
    node_index = {}
    for i in range(len(symbol_list)):
        node_index[symbol_list[i]] = i

    # index로 node_list 생성
    node_list = node_index.values()

    # index로 edge_list 생성
    edge_list = network_data[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']].values.tolist()  #
    for i in range(len(edge_list)):
        edge_list[i][0] = node_index.get(edge_list[i][0])
        edge_list[i][1] = node_index.get(edge_list[i][1])
    
    # 무방향 그래프 오브젝트 생성
    G = nx.Graph()

    # 그래프에 노드 추가
    G.add_nodes_from(node_list)

    # 그래프에 엣지 추가
    G.add_edges_from(edge_list)

    print(f'Number of created nodes: {G.number_of_nodes()}')
    print(f'Number of created edges: {G.number_of_edges()}')

    return G, node_index

def RWR_initializing(G, seed_nodes, indirect_target_nodes, is_weighted=False):
    norm_A = np.zeros(shape=(len(G), len(G)))
    if is_weighted:  
        for i, neighbor_dict in G.adjacency():
            for j, v in neighbor_dict.items():
                norm_A[i][j] = v.get("weight", 1 / len(neighbor_dict))
    else:  
        for i, neighbor_dict in G.adjacency():  
            for j, v in neighbor_dict.items():  
                norm_A[i][j] = 1 / len(neighbor_dict)
    
    # seed nodes와 indirect target nodes를 모두 포함한 딕셔너리 생성
    personalization = {node: 1 for node in seed_nodes}
    for node in indirect_target_nodes:
        personalization[node] = 0.3  
        
    r_0 = np.array([personalization.get(n, 0) for n in range(len(G))]) 
    r_c = np.repeat(1 / len(G), len(G)) 
    return {"norm_A": norm_A, "r_0": r_0, "r_c": r_c, "N": len(G)}

def RWR(initialized, prob=0.85, max_iter=100, tol=1.0e-6):
    if initialized is None:
        raise ValueError('initialized information must be required')

    norm_A = initialized['norm_A']  # 인접행렬 A (정규화된)
    r_0 = initialized['r_0']        # 시작 노드
    r_c = initialized['r_c']        # 가중치 행렬
    N = initialized['N']            # 노드의 개수
    for iteration in range(max_iter):
        r_prev = r_c
        r_c = prob * r_c @ norm_A + (1 - prob) * r_0  # RWR algorithm
        err = np.absolute(r_c - r_prev).sum()
        if err < N * tol:
            print(f'RWR iteration = {iteration + 1}')
            print('Converged')
            return r_c
        else:
            print(f'RWR iteration = {iteration + 1}, Iteration until convergence ...')
            print(f'{err} -> {N * tol}')
    return "NotConverged"

# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()

# BIOGRID-Homo_sapiens Data 기반 PPI Network 구축
network_data = pd.read_csv("/data/home/sss2061/식약처/Data/BIOGRID-ORGANISM-Homo_sapiens.tab3.txt", sep="\t", encoding="cp949")  
G, node_index = create_ppi_network(network_data=network_data)  
entrez_list = list(node_index.keys())  

# 네트워크 활용 gene_phenotype 정보 생성
gene_phenotype_file = '/data/home/sss2061/식약처/gene_2023_phenotype_info.csv'
if os.path.isfile(gene_phenotype_file): 
    gene_phenotype_df = pd.read_csv(gene_phenotype_file, encoding='UTF-8', converters={'Phenotype': ast.literal_eval}) 
else: 
    gene_phenotype_info = {}
    for g in range(len(entrez_list)):
        conn_cur.execute(f"with tmp1 as (select g.geneid from gene g where g.entrezid = '{entrez_list[g]}'),"
                         f"tmp2 as (select gp.phenid from gene_phenotype gp, tmp1 where gp.geneid = tmp1.geneid)"
                         f"select distinct p.name from phenotype p, tmp2 where p.phenid = tmp2.phenid;")
        # TASS DB 기반 gene-phenotype 정보 추출
        selected_phenotype = conn_cur.fetchall()
        phenotypes = []
        for i in range(len(selected_phenotype)):
             phenotypes.append(selected_phenotype[i][0]) 
        gene_phenotype_info[entrez_list[g]] = phenotypes
        print(f'{g + 1}/{len(entrez_list)}')
    gene_phenotype_mapping = [[entrez_list[i], gene_phenotype_info.get(entrez_list[i])] for i in range(len(entrez_list))]
    gene_phenotype_df = pd.DataFrame(data=gene_phenotype_mapping, columns=['Entrez ID', 'Phenotype'])
    gene_phenotype_df.to_csv(gene_phenotype_file, index=False, encoding='UTF-8')

phenotype_list = []
for phen_list in gene_phenotype_df['Phenotype']:
    for phen in phen_list:
        if phen not in phenotype_list:
            phenotype_list.append(phen)

Compound_list=['CP00002311','CP00056186','CP00307121'] #사용자가 입력하는 compound list

global directed_target_gene_list
directed_target_gene_list=[]  #사용자가 입력한 compound의 direct target gene 개수가 들어있는 list

global indirected_target_gene_list_count
indirected_target_gene_list_count=[] #사용자가 입력한 compound의 indirect target gene 개수가 들어있는 list

rwr_mapping_entrez_list=[] #사용자가 입력한 compound 별 rwr mapping entrez가 들어있는 list

for c in range(len(Compound_list)):
    
    compoundid1=Compound_list[c]
    
    rwr_mapping_entrez=compound_target(compoundid1)
    rwr_mapping_entrez_list.append(rwr_mapping_entrez)

rwr_mapping_entrez2 = combine_lists(rwr_mapping_entrez_list) #rwr mapping 조화평균 추출

rwr_result = pd.DataFrame(data=rwr_mapping_entrez2, columns=['Entrez ID', 'RWR_score'])
rwr_result = pd.merge(rwr_result, gene_phenotype_df, how='outer')
rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[
    rwr_result['Phenotype'].isnull()].apply(lambda x: [])
rwr_result = rwr_result.sort_values(by='RWR_score', ascending=False)

# phenotype 별 RWR score를 계산하기 위한 딕셔너리 생성
phenotype_rwr_score_dict = {}
for phen in phenotype_list:
    phenotype_rwr_score_dict[phen] = 0    
    
# 각 phenotype 별 RWR score 확인
for i in range(len(rwr_result)):
    rwr_score = rwr_result['RWR_score'].iloc[i]
    if rwr_result['Phenotype'].iloc[i]:
        for phen in rwr_result['Phenotype'].iloc[i]:
            phenotype_rwr_score_dict[phen] += rwr_score

Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

#network score. csv 파일로 저장
inferred_phenotype2 = pd.DataFrame(data=[pair for pair in zip(list(phenotype_rwr_score_dict.keys()),
                                                             list(phenotype_rwr_score_dict.values()))],
                                  columns=['Phenotype', 'Network_score'])
inferred_phenotype2 = inferred_phenotype2.sort_values(by='Network_score', ascending=False)

# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
inferred_phenotype2.to_csv('3_network_score.csv',index=False, encoding='UTF-8')

#----------------------------------------------------------------1,000
output_dict={}
print("확인용 바깥",indirected_target_gene_list_count) #확인용
for h in range(1000):
    p_rwr_mapping_entrez_list=[] #1번당 compound별 random target gene rwr mapping list
    
    print(h,"번째 상황") # 제거
    for k in range(len(Compound_list)):
        rwr_mapping_entrez=rwr_compound_target(k,entrez_list)
        p_rwr_mapping_entrez_list.append(rwr_mapping_entrez)
    
    rwr_mapping_entrez3 = combine_lists(p_rwr_mapping_entrez_list) #rwr mapping 조화평균 추출
    
    rwr_result = pd.DataFrame(data=rwr_mapping_entrez3, columns=['Entrez ID', 'RWR_score'])
    rwr_result = pd.merge(rwr_result, gene_phenotype_df, how='outer')
    rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()].apply(lambda x: [])
    rwr_result = rwr_result.sort_values(by='RWR_score', ascending=False)
    
# phenotype 별 RWR score를 계산하기 위한 딕셔너리 생성
    phenotype_rwr_score_dict = {}
    for phen in phenotype_list:
        phenotype_rwr_score_dict[phen] = 0

# 각 phenotype 별 RWR score 확인
    for i in range(len(rwr_result)):
        rwr_score = rwr_result['RWR_score'].iloc[i]
        if rwr_result['Phenotype'].iloc[i]:
            for phen in rwr_result['Phenotype'].iloc[i]:
                phenotype_rwr_score_dict[phen] += rwr_score

    key_list=list(phenotype_rwr_score_dict.keys())
    for i in key_list:
        if i not in output_dict:
            output_dict[i]=list([phenotype_rwr_score_dict[i]])
        else:
            output_dict[i].append(phenotype_rwr_score_dict[i])
print("rwr 종료")

#rwr. score csv 파일로 저장
inferred_phenotype3 = pd.DataFrame(data=[pair for pair in zip(list(output_dict.keys()),
                                                             list(output_dict.values()))],
                                  columns=['Phenotype', 'Network_score'])
inferred_phenotype3 = inferred_phenotype3.sort_values(by='Network_score', ascending=False)

inferred_phenotype3.to_csv('3_P_value.csv',index=False, encoding='UTF-8')

P_value_Phenotype=list(output_dict.keys())
P_value_Score=list(output_dict.values())

#----------------------------------------------------------------z-score  
count=0
z_score_list=[]
outlier_list=[] #제거가능
P_value_list=[]
rank_number=0

for i in range(len(P_value_Score)):
    Numberlist=P_value_Score[i].copy()
    Numberlist.append(Phenotype_score_Score[i])

    # 평균 계산
    mean = calculate_mean(Numberlist)

    # 표준편차 계산
    std_deviation = calculate_standard_deviation(Numberlist)

    z_score=(Phenotype_score_Score[i]-mean)/std_deviation
    z_score_list.append(z_score)
    if z_score >=-2.58 and z_score<=2.58: #제거가능
        outlier=1
        count=count+1
    else:
        outlier=0 
    outlier_list.append(outlier) #제거가능

    Numberlist.sort(reverse=True)

    rank=Numberlist.index(Phenotype_score_Score[i]) #순위찾기
    if rank<5:
        rank_number=rank_number+1
    P_value_list.append(rank)


#------------------------------------
phenotype_umlsid = []
# phenotype_list를 순회하면서 각 이름에 대한 UMLS ID를 가져옴
for phen_name in Phenotype_score_Phenotype:
    # 이름에 단일 따옴표가 포함된 경우 이스케이프
    escaped_name = phen_name.replace("'", "''")
    
    conn_cur.execute(f"SELECT umlsid FROM phenotype WHERE name = '{escaped_name}'")
    umlsid_result = conn_cur.fetchone()
    
    # UMLS ID가 존재하면 리스트에 추가
    if umlsid_result:
        phenotype_umlsid.append(umlsid_result[0])
    else:
        # UMLS ID가 없는 경우에 대한 처리 (예: None 또는 다른 기본값)
        phenotype_umlsid.append(None)

# 결과를 DataFrame으로 만들어 저장
phenotype_umlsid_df = pd.DataFrame({'Name': Phenotype_score_Phenotype, 'UMLSID': phenotype_umlsid})
print(phenotype_umlsid_df)

#------------------------

#     Phenotype_score['Phenotype']=Phenotype_score_Phenotype
#     Phenotype_score['Network_score']=Phenotype_score_Score
#     Phenotype_score['z-score']=z_score_list
#     Phenotype_score['outlier']=outlier_list

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
Phenotype_score = pd.DataFrame({
    'Phenotype': Phenotype_score_Phenotype,
    'Network_score': Phenotype_score_Score,
    'z-score': z_score_list,
    'P-value': P_value_list,
    'UMLSID': phenotype_umlsid
})

Phenotype_score=Phenotype_score.sort_values(by='P-value',ascending=True)

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
Phenotype_score.to_csv('3_검증.csv',index=False, encoding='UTF-8')
    
end = time.time()
sec = (end - start)
result = datetime.timedelta(seconds=sec)
print("걸린 시간 : ",result)
print(f"걸린 시간 : {end - start:.5f} sec")

/tmp/ipykernel_284961/2755789250.py:234: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  network_data = pd.read_csv("/data/home/sss2061/식약처/Data/BIOGRID-ORGANISM-Homo_sapiens.tab3.txt", sep="\t", encoding="cp949")


Number of created nodes: 27421
Number of created edges: 817463
Compound CP00002311 Target gene 개수 211
Compound CP00002311 indirected_Target gene 개수 992
RWR iteration = 1, Iteration until convergence ...
324.54528978836083 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
64.59058986369786 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
12.915827911507916 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
2.583042525096972 -> 0.027420999999999997
RWR iteration = 5, Iteration until convergence ...
0.5166084667955233 -> 0.027420999999999997
RWR iteration = 6, Iteration until convergence ...
0.10332169335910199 -> 0.027420999999999997
RWR iteration = 7
Converged
Compound CP00056186 Target gene 개수 10
Compound CP00056186 indirected_Target gene 개수 278
RWR iteration = 1, Iteration until convergence ...
72.43008759824237 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
14.188412111641004 -> 0.0

/tmp/ipykernel_284961/2755789250.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[


확인용 바깥 [992, 278, 182]
0 번째 상황
RWR iteration = 1, Iteration until convergence ...
407.7296654660653 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
81.23161395654111 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
16.243280896069585 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
3.2486425250969684 -> 0.027420999999999997
RWR iteration = 5, Iteration until convergence ...
0.6497284667954858 -> 0.027420999999999997
RWR iteration = 6, Iteration until convergence ...
0.12994569335909506 -> 0.027420999999999997
RWR iteration = 7
Converged
RWR iteration = 1, Iteration until convergence ...
75.62652055453844 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
14.852345502631561 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
2.957254626962324 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
0.5913626826308064 -> 0.027420999999999997
RWR 

/tmp/ipykernel_284961/2755789250.py:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()].apply(lambda x: [])


...

998 번째 상황
RWR iteration = 1, Iteration until convergence ...
407.72991344805956 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
81.22934068851234 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
16.243260022642716 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
3.2486418671820676 -> 0.027420999999999997
RWR iteration = 5, Iteration until convergence ...
0.6497283734364379 -> 0.027420999999999997
RWR iteration = 6, Iteration until convergence ...
0.12994567468728652 -> 0.027420999999999997
RWR iteration = 7
Converged
RWR iteration = 1, Iteration until convergence ...
75.62680332827917 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
14.84504388640633 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
2.9572117280079615 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
0.5913623984855899 -> 0.027420999999999997
RWR iteration = 5, Itera

/tmp/ipykernel_284961/2755789250.py:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()].apply(lambda x: [])


999 번째 상황
RWR iteration = 1, Iteration until convergence ...
407.72908643584896 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
81.22971588072299 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
16.243278592126934 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
3.2486425828319905 -> 0.027420999999999997
RWR iteration = 5, Iteration until convergence ...
0.6497284667955209 -> 0.027420999999999997
RWR iteration = 6, Iteration until convergence ...
0.12994569335911213 -> 0.027420999999999997
RWR iteration = 7
Converged
RWR iteration = 1, Iteration until convergence ...
75.62696348104491 -> 0.027420999999999997
RWR iteration = 2, Iteration until convergence ...
14.841689525448531 -> 0.027420999999999997
RWR iteration = 3, Iteration until convergence ...
2.957129346032171 -> 0.027420999999999997
RWR iteration = 4, Iteration until convergence ...
0.5913628001000623 -> 0.027420999999999997
RWR iteration = 5, Itera

/tmp/ipykernel_284961/2755789250.py:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()] = rwr_result['Phenotype'].loc[rwr_result['Phenotype'].isnull()].apply(lambda x: [])


rwr 종료
                                                   Name    UMLSID
0                          Abnormal auditory perception  C0375257
1                              abnormal pulmonary shunt  C0598223
2     Abnormal uterine bleeding due to endometrial d...  C5190722
3                        abuse; hormones, non-dependent  C1404082
4                     Accidental acetylcholine overdose  C0573239
...                                                 ...       ...
4035        Fracture of base of skull, unspecified side  C4269243
4036  Cataract (lens) fragments in eye following cat...  C2881457
4037  Subluxation of other parts of right shoulder g...  C2842474
4038             infestation by Clinostomum complanatum  C5389715
4039                     Bilateral age-related cataract  C2025430

[4040 rows x 2 columns]
걸린 시간 :  3:10:37.834529
걸린 시간 : 11437.83453 sec
